# Table of Contents
* [Exploring science textbooks for parsing and annotations](#Exploring-science-textbooks-for-parsing-and-annotations)
	* [basic parameters](#basic-parameters)
	* [checking extractability](#checking-extractability)
	* [Testing pdf miner on single page](#Testing-pdf-miner-on-single-page)
		* [drawing bounding boxes over image](#drawing-bounding-boxes-over-image)
	* [Drawing multiple pages](#Drawing-multiple-pages)
* [END](#END)


In [264]:
%%capture
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [2]:
from wand.image import Image as WImage
from IPython.display import display
import PIL.Image as Image
import cv2

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.converter import TextConverter

In [265]:
from pdf_processing import draw_pdf_with_boxes

# Exploring science textbooks for parsing and annotations

## basic parameters

In [293]:
ls pdfs/ | wc -l 

In [294]:
book_list = !ls pdfs/
book_list

There are 28 total. With a couple of series-

In [313]:
book_breakdowns = defaultdict(list)

In [314]:
spectrum_science =  !ls pdfs/ | grep 'Spectrum Science'
book_breakdowns['spectrum_sci'] = spectrum_science
print('Spectrum Science,: ', len(spectrum_science), ' total')
# print('\n'.join(spectrum_science))

In [315]:
daily_science =  !ls pdfs/ | grep 'Daily Sc' 
book_breakdowns['daily_sci'] = daily_science
print('Daily Science: ', len(daily_science), ' total')
# print('\n'.join(daily_science))

In [316]:
read_understand =  !ls pdfs/ | grep 'Read and Understand Science' 
book_breakdowns['read_und_sci'] = read_understand
print('Read and Understand Science: ', len(read_understand), ' total')
# print('\n'.join(read_understand))

In [320]:
workbooks =  !ls pdfs/ | grep -i  'workbook' 
book_breakdowns['workbooks'] = workbooks
print('workbooks: ', len(workbooks), ' total')
# print('\n'.join(workbooks))

In [318]:
for book in book_list:
    if not sum([book in series for series in book_breakdowns.values()]):
        book_breakdowns['misc'].append(book)

In [319]:
sum([len(books) for books in book_breakdowns.values()])

all books accounted for in groupings

In [321]:
for group, books in book_breakdowns.items():
    print(group)
    print('\n'.join(books + [' ']))

## checking extractability 

In [15]:
extractable = 0
for textbook in book_list:
    test_book_path = './pdfs/' + textbook
    with open(test_book_path, 'r') as fp:
        parser = PDFParser(fp)
        document = PDFDocument(parser)
        extractable += document.is_extractable
extractable

all of the documents are extractable!

In [228]:
!find ./pdfs/* | xargs -L 1 pdffonts

fonts and encodings are consistent

## Testing pdf miner on single page

In [16]:
pages = []
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
    # receive the LTPage object for the page.
    pages.append(device.get_result())

In [17]:
# test_book = book_breakdowns['daily_sci'][3]
test_book ='test_page.pdf'
test_book_path = './' + test_book
test_book

### drawing bounding boxes over image

In [85]:
page_file ='test_page.pdf'
page_layout = make_page_layout(page_file)
page_png_stream, y_height = make_png_stream(page_file)
page_img = make_open_cv_img(page_png_stream)

for box in page_layout._objs:
    lr, ul = get_bbox_tuple(box, y_height)
    cv2.rectangle(page_img, ul, lr, color=random_color(), thickness=2)
    try:
        print(box.get_text())
    except AttributeError:
        pass
display(Image.fromarray(page_img, 'RGB'))

In [ ]:
 laparams = LAParams()
    page_layouts = []
    with open(pdf_file, 'r') as fp:
        parser = PDFParser(fp)
        document = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layout = device.get_result()
            page_layouts.append(layout)
    return page_layouts

pdf miner doesn't seem to find figs/ images

## Drawing multiple pages

In [325]:
for group, books in book_breakdowns.items():
    print(group)
    print('\n'.join(books[3:4] + [' ']))

In [326]:
book_file ='./pdfs/' + book_breakdowns['misc'][2]
draw_pdf_with_boxes(book_file, [50,53])

# END